In [1]:
import requests

In [13]:
r = requests.get("https://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?table=cumulative")

In [14]:
f = open('planet_data.csv', 'w')
f.write(r.text)

3179031

In [15]:
import pandas as pd
df = pd.read_csv('planet_data.csv')
print(df.columns)

Index(['kepid', 'kepoi_name', 'kepler_name', 'koi_disposition',
       'koi_pdisposition', 'koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss',
       'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_period_err1',
       'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1',
       'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1', 'koi_impact_err2',
       'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth',
       'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 'koi_prad_err1',
       'koi_prad_err2', 'koi_teq', 'koi_teq_err1', 'koi_teq_err2', 'koi_insol',
       'koi_insol_err1', 'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num',
       'koi_tce_delivname', 'koi_steff', 'koi_steff_err1', 'koi_steff_err2',
       'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad',
       'koi_srad_err1', 'koi_srad_err2', 'ra_str', 'dec_str', 'koi_kepmag',
       'koi_kepmag_err'],
      dtype='object')


In [5]:
confirmed = 0
candidate = 0
false_positive = 0
extracted_data = list()
for index, planet in df.iterrows():
    
    # attributes we care about
    disposition = planet.loc['koi_disposition'] # y data
    
    # x data
    koi_period = planet.loc['koi_period']
#     koi_eccen = planet.loc['koi_eccen']
#     koi_longp = planet.loc['koi_longp']
    koi_impact = planet.loc['koi_impact']
    koi_duration = planet.loc['koi_duration']
    koi_depth = planet.loc['koi_depth']
    koi_prad = planet.loc['koi_prad']
    koi_teq = planet.loc['koi_teq']
    
    planet_info = [koi_period, koi_impact, koi_duration, koi_depth, koi_prad, koi_teq]
    
    if disposition == 'CONFIRMED':
        extracted_data.append((planet_info, 1))
    elif disposition == 'FALSE POSITIVE':
        extracted_data.append((planet_info, 0))


In [6]:
import numpy as np

clean_data = []

for planet in extracted_data:
    array = np.array(planet[0])
    if np.isnan(array).any():
        pass
    else:
        clean_data.append(planet)

np.random.shuffle(clean_data)

split = int(0.8 * len(clean_data))
train_data = clean_data[:split]
test_data = clean_data[split:]

x_train = np.array([planet[0] for planet in train_data])
y_train = np.array([planet[1] for planet in train_data])

x_test = np.array([planet[0] for planet in test_data])
y_test = np.array([planet[1] for planet in test_data])

In [7]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier([100, 100, 100, 100, 100, 100])
mlp.fit(x_train, y_train)
score = mlp.score(x_test, y_test)
print(score)

0.7976961843052556


In [8]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(x_train, y_train)
score = clf.score(x_test, y_test)
print(score)

0.6594672426205903


In [9]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(x_train, y_train)
score = gnb.score(x_test, y_test)
print(score)

0.7134629229661628


In [10]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf.fit(x_train, y_train)
score = clf.score(x_test, y_test)
print(score)
r = tree.export_text(clf, feature_names=['Period', 'Impact', 'Duration', 'Depth', 'Prad', 'Teq'])
# print(r)

0.8473722102231821


In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

X = np.array([planet[0] for planet in clean_data])
y = np.array([planet[1] for planet in clean_data])

clf = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X, y, cv=10)
scores.mean()

0.8588903862500779